In [41]:
import numpy as np
import pandas as pd
import sklearn as sk

In [42]:
dataloc = "data"
carhacking = "Car_Hacking_Challenge_Dataset_rev20Mar2021"
prelim = "0_Preliminary"
training = "0_Training"
filename_1 = "Pre_train_D_1.csv"

In [43]:
import os

prelim_train_dir = os.path.join("..", dataloc, "raw", carhacking, prelim, training)
csv1 = os.path.join(prelim_train_dir, filename_1)

In [71]:
df = pd.read_csv(csv1, encoding='unicode_escape')
df.head()

,Timestamp,Arbitration_ID,DLC,Data,Class,SubClass
0,1.597760e+09,153,8,20 A1 10 FF 00 FF 50 1F,Normal,Normal
1,1.597760e+09,220,8,13 24 7F 60 05 FF BF 10,Normal,Normal
2,1.597760e+09,507,4,08 00 00 01,Normal,Normal
3,1.597760e+09,356,8,00 00 00 80 16 00 00 00,Normal,Normal
4,1.597760e+09,340,8,FC 03 00 E4 B7 21 FA 3C,Normal,Normal


In [72]:
df[["d1", "d2", "d3", "d4", "d5", "d6", "d7", "d8"]] = df.Data.str.split(" ", expand=True)
df.head()

,Timestamp,Arbitration_ID,DLC,Data,Class,SubClass,d1,d2,d3,d4,d5,d6,d7,d8
0,1.597760e+09,153,8,20 A1 10 FF 00 FF 50 1F,Normal,Normal,20,A1,10,FF,00,FF,50,1F
1,1.597760e+09,220,8,13 24 7F 60 05 FF BF 10,Normal,Normal,13,24,7F,60,05,FF,BF,10
2,1.597760e+09,507,4,08 00 00 01,Normal,Normal,08,00,00,01,None,None,None,None
3,1.597760e+09,356,8,00 00 00 80 16 00 00 00,Normal,Normal,00,00,00,80,16,00,00,00
4,1.597760e+09,340,8,FC 03 00 E4 B7 21 FA 3C,Normal,Normal,FC,03,00,E4,B7,21,FA,3C


In [73]:
df["d3"].isna().sum()

1234

In [74]:
df["d1_int"] = df.apply(lambda x: 999 if x["d1"] is None else int(x["d1"], 16), axis=1)
df["d2_int"] = df.apply(lambda x: 999 if x["d2"] is None else int(x["d2"], 16), axis=1)
df["d3_int"] = df.apply(lambda x: 999 if x["d3"] is None else int(x["d3"], 16), axis=1)
df["d4_int"] = df.apply(lambda x: 999 if x["d4"] is None else int(x["d4"], 16), axis=1)

df["d5_int"] = df.apply(lambda x: 999 if x["d5"] is None else int(x["d5"], 16), axis=1)
df["d6_int"] = df.apply(lambda x: 999 if x["d6"] is None else int(x["d6"], 16), axis=1)
df["d7_int"] = df.apply(lambda x: 999 if x["d7"] is None else int(x["d7"], 16), axis=1)
df["d8_int"] = df.apply(lambda x: 999 if x["d8"] is None else int(x["d8"], 16), axis=1)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806390 entries, 0 to 806389
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Timestamp       806390 non-null  float64
 1   Arbitration_ID  806390 non-null  object 
 2   DLC             806390 non-null  int64  
 3   Data            806390 non-null  object 
 4   Class           806390 non-null  object 
 5   SubClass        806390 non-null  object 
 6   d1              806390 non-null  object 
 7   d2              806390 non-null  object 
 8   d3              805156 non-null  object 
 9   d4              805156 non-null  object 
 10  d5              743095 non-null  object 
 11  d6              727685 non-null  object 
 12  d7              695249 non-null  object 
 13  d8              663856 non-null  object 
 14  d1_int          806390 non-null  int64  
 15  d2_int          806390 non-null  int64  
 16  d3_int          806390 non-null  int64  
 17  d4_int    

In [76]:
X = df[["d1_int", "d2_int", "d3_int", "d4_int", "d5_int", "d6_int", "d7_int", "d8_int"]].to_numpy()
X

array([[ 32, 161,  16, ..., 255,  80,  31],
       [ 19,  36, 127, ..., 255, 191,  16],
       [  8,   0,   0, ..., 999, 999, 999],
       ...,
       [  0,   0,   0, ...,  40,  11,  66],
       [  4, 127, 255, ..., 123,   0,  38],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [78]:
import math

def build_nonoverlapping_sequence(X, seq_num=4):
    seq_len = X.shape[1] * seq_num
    n = math.floor(X.shape[0] / seq_len)
    r = X.shape[0] % seq_len
    if r != 0:
        # Cut off not divisible part
        seqs = X[:-r].reshape(-1,32)
    else:
        seqs = X.reshape(-1,32)

    return seqs

In [79]:
Xnew = build_nonoverlapping_sequence(X, 32)
Xnew.shape

(201536, 32)

In [81]:
from abc import ABC, abstractmethod

class UnsupervisedEmbedding(ABC):
    def __init__(self, embedding_root: str = None, embedding_for: str = None,
                 embedding_dim: int = 100, embedding_version: float = 1.0):
        self.embedding_root = embedding_root
        self.embedding_for = embedding_for
        self.embedding_dim = embedding_dim
        self.embedding_version = embedding_version

    @abstractmethod
    def fit(self, X):
        pass

In [82]:
import fasttext

In [83]:
def generate_model_file_path(embedding_root: str = None, embedding_for: str = None,
                             embedding_model='skipgram', embedding_wordNgrams: int = 1,
                             embedding_dim: int = 100, train_seq_len: int = 10, embedding_version: float = 1.0,
                             embedding_type='fasttext'):
    filename = f"{embedding_for}_{embedding_type}_{embedding_model}_" \
               f"{embedding_wordNgrams}wordNgram_{embedding_dim}dim_{train_seq_len}trainseq_v{embedding_version}.bin"
    print(os.path.join(embedding_root, filename))
    return os.path.join(embedding_root, filename)

In [84]:
class FastTextEmbedding(UnsupervisedEmbedding):
    def __init__(self, embedding_root: str = None, embedding_for: str = None,
                 embedding_model='skipgram', embedding_wordNgrams: int = 1,
                 embedding_dim: int = 100, embedding_version: float = 1.0,
                 epochs: int = 10, minCount: int = 1, maxn: int = 0):
        super().__init__(embedding_root, embedding_for, embedding_dim, embedding_version)
        self.embedding_model = embedding_model

        # Can use wordN grams by setting 2
        # https://fasttext.cc/docs/en/supervised-tutorial.html
        self.embedding_wordNgrams = embedding_wordNgrams

        self.epochs = epochs
        self.minCount = minCount
        self.maxn = maxn

        self.embedding_config = {
            'embedding_root': self.embedding_root,
            'embedding_for': self.embedding_for, 'embedding_model': self.embedding_model,
            "embedding_wordNgrams": self.embedding_wordNgrams, 'embedding_dim': self.embedding_dim,
            'embedding_version': self.embedding_version, 'embedding_type': 'fasttext'
        }

    def fit(self, X):
        seq_len = X.shape[1]
        # if X is not None:
        #     seq_len = len(_tokenize_by_spaces(X[0])) 
        # print(f"Calc BBBBBBBBBB {seq_len} , X.shape[1] = {X.shape[1]}")


        data_temp_file_path = FastTextEmbedding.generate_temp_seq_storage_file_path(self.embedding_for)
        np.savetxt(data_temp_file_path, X.astype(int), fmt='%i')

        # Create embeddings for event id https://fasttext.cc/docs/en/python-module.html
        fasttext_model = fasttext.train_unsupervised(data_temp_file_path,
                                                     model=self.embedding_model,
                                                     dim=self.embedding_dim,
                                                     wordNgrams=self.embedding_wordNgrams,
                                                     epoch=self.epochs, minCount=self.minCount, maxn=self.maxn)
        cfg_copy = self.embedding_config.copy()
        cfg_copy["train_seq_len"] = seq_len
        model_file_path = generate_model_file_path(**cfg_copy)
        fasttext_model.save_model(model_file_path)

        os.remove(data_temp_file_path)

        # print(fasttext_model.get_words())
        # word_embeddings = model.get_output_matrix()
        # print(word_embeddings)

        return fasttext_model
    
    @staticmethod
    def generate_temp_seq_storage_file_path(embedding_for=None):
        return embedding_for + '_eventid_token_seq.txt'

In [85]:
fastText_embedding_cfg = {
    'embedding_root': "C:/Users/jvana/carhacking/features",
    'embedding_for': "Car_Hacking_Challenge_Dataset_rev20Mar2021", 'embedding_model': "skipgram",
    "embedding_wordNgrams": 3, 'embedding_dim': 100,
    'embedding_version': "1.0"
}

fasttext_embedding = FastTextEmbedding(**fastText_embedding_cfg)
fasttext_model = fasttext_embedding.fit(Xnew) # fits and saves model

C:/Users/jvana/carhacking/features\Car_Hacking_Challenge_Dataset_rev20Mar2021_fasttext_skipgram_3wordNgram_100dim_32trainseq_v1.0.bin


In [86]:
def fast_text_from_model_file(embedding_root: str = None, embedding_for: str = None,
                              embedding_model='skipgram', embedding_wordNgrams: int = 1,
                              embedding_dim: int = 100, train_seq_len: int = 10, embedding_version: float = 1.0):
    model_file_path = generate_model_file_path(embedding_root, embedding_for, embedding_model,
                                               embedding_wordNgrams, embedding_dim, train_seq_len,
                                               embedding_version, 'fasttext')
    model = fasttext.load_model(model_file_path)
    return model

In [87]:
fastText_embedding_cfg = {
    'embedding_root': "C:/Users/jvana/carhacking/features",
    'embedding_for': "Car_Hacking_Challenge_Dataset_rev20Mar2021", 'embedding_model': "skipgram",
    "embedding_wordNgrams": 3, 'embedding_dim': 100,
    'embedding_version': "1.0", "train_seq_len": 32
}

fasttext_model = fast_text_from_model_file(**fastText_embedding_cfg)
word_embeddings = np.array([fasttext_model.get_word_vector(str(word_token))
                            for word_token in np.arange(0,255)])


C:/Users/jvana/carhacking/features\Car_Hacking_Challenge_Dataset_rev20Mar2021_fasttext_skipgram_3wordNgram_100dim_32trainseq_v1.0.bin


In [88]:
word_embeddings

array([[ 0.01350737, -0.03246043,  0.1353021 , ...,  0.03284965,
         0.20701405, -0.00108776],
       [-0.02180098, -0.07381336,  0.08867939, ..., -0.03638043,
        -0.08245512, -0.01660034],
       [ 0.02409102, -0.03610906,  0.00647557, ..., -0.05488613,
        -0.06102106,  0.23384662],
       ...,
       [-0.12718815,  0.22688039,  0.08068047, ...,  0.11819232,
         0.42247272, -0.3135497 ],
       [-0.08785516, -0.02640463,  0.11216791, ..., -0.09137792,
         0.1770951 ,  0.02200276],
       [-0.07057732,  0.34931365,  0.27632374, ..., -0.1625063 ,
         0.24944316, -0.03379932]], dtype=float32)

In [89]:
def generate_word2vec_model_file_path(embedding_root: str = None, embedding_for: str = None,
                             embedding_model='skipgram', embedding_wordNgrams: int = 1,
                             embedding_dim: int = 100, train_seq_len: int = 10, embedding_version: float = 1.0,
                             embedding_type='fasttext'):
    filename = f"{embedding_for}_{embedding_type}_{embedding_model}_" \
               f"{embedding_wordNgrams}wordNgram_{embedding_dim}dim_{train_seq_len}trainseq_v{embedding_version}.word2vec"
    return os.path.join(embedding_root, filename)

In [100]:
def generate_word2vec_model_bin_file_path(embedding_root: str = None, embedding_for: str = None,
                             embedding_model='skipgram', embedding_wordNgrams: int = 1,
                             embedding_dim: int = 100, train_seq_len: int = 10, embedding_version: float = 1.0,
                             embedding_type='fasttext'):
    filename = f"{embedding_for}_{embedding_type}_{embedding_model}_" \
               f"{embedding_wordNgrams}wordNgram_{embedding_dim}dim_{train_seq_len}trainseq_v{embedding_version}.bin"
    return os.path.join(embedding_root, filename)

In [101]:
from gensim.models import Word2Vec, KeyedVectors

fastText_embedding_cfg = {
    'embedding_root': "C:/Users/jvana/carhacking/features",
    'embedding_for': "Car_Hacking_Challenge_Dataset_rev20Mar2021", 'embedding_model': "skipgram",
    "embedding_wordNgrams": 3, 'embedding_dim': 100,
    'embedding_version': "1.0", "train_seq_len": 32
}

model_bin_file_path = generate_word2vec_model_bin_file_path(**fastText_embedding_cfg)

In [102]:
model_bin_file_path

'C:/Users/jvana/carhacking/features\\Car_Hacking_Challenge_Dataset_rev20Mar2021_fasttext_skipgram_3wordNgram_100dim_32trainseq_v1.0.bin'

In [105]:
wv = KeyedVectors.load_word2vec_format(model_bin_file_path, binary=True)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 0: invalid start byte

In [95]:
from gensim.models import Word2Vec

# Load the Word2Vec model
model = Word2Vec.load(model_bin_file_path)

# Use the model for various NLP tasks


UnpicklingError: invalid load key, '\xba'.